In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [4]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [6]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 82, 20)
non_lin = [1, 2]
non_lin_dict = {1: "sigmoid", 2: "relu"}

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = alpha)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 150) 
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))    
            print "new best model for cell", c, ". r2 test: ", r2_test

0%      100%
[#         ] | ETA: 00:00:00

 testing non-linearity:  sigmoid ... testing  1  pc's...
new best model for cell  517442753 . r2 test:  0.00210441042664
new best model for cell  517444738 . r2 test:  0.00641274508731
new best model for cell  517443331 . r2 test:  -0.0549593256823
new best model for cell  517442436 . r2 test:  -0.00755922771403
new best model for cell  517445575 . r2 test:  -0.0255365223836
new best model for cell  517444426 . r2 test:  -0.0155938155311
new best model for cell  517442471 . r2 test:  -0.0141746872328
new best model for cell  517443181 . r2 test:  0.0346951701921
new best model for cell  517443087 . r2 test:  -0.0402827941974
new best model for cell  517446833 . r2 test:  0.0285863042242
new best model for cell  517443827 . r2 test:  0.0121207590724
new best model for cell  517443924 . r2 test:  -0.037076681654
new best model for cell  517443957 . r2 test:  0.0260458658661
new best model for cell  517442722 . r2 test:  -0.0249313804947
new best model for cell  517446488 . r2 test:  -0.0

[##        ] | ETA: 00:02:52

 517443571 . r2 test:  0.04091197097
testing non-linearity:  sigmoid ... testing  21  pc's...
new best model for cell  517444738 . r2 test:  0.0242196818934
new best model for cell  517443331 . r2 test:  0.00157091122544
new best model for cell  517445575 . r2 test:  0.0260771402207
new best model for cell  517444426 . r2 test:  0.00613111563461
new best model for cell  517443087 . r2 test:  0.00743891588946
new best model for cell  517443924 . r2 test:  0.024279751278
new best model for cell  517442722 . r2 test:  0.0170791069371
new best model for cell  517446488 . r2 test:  -0.00514540843774
new best model for cell  517442675 . r2 test:  0.00752513386415
new best model for cell  517444175 . r2 test:  0.0307689908106
new best model for cell  517447069 . r2 test:  -0.0124956665145
testing non-linearity: 

[###       ] | ETA: 00:05:43

...

[######### ] | ETA: 00:03:33

 relu ... testing  61  pc's...
testing non-linearity: 

[##########] | ETA: 00:00:00

 relu ... testing  81  pc's...



Total time elapsed: 00:40:47


In [8]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
    print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n

explained var:  0.025 non_lin: 1 n_components 61
explained var:  0.024 non_lin: 1 n_components 21
explained var:  0.041 non_lin: 1 n_components 81
explained var:  0.025 non_lin: 1 n_components 81
explained var:  0.028 non_lin: 1 n_components 61
explained var:  0.007 non_lin: 1 n_components 41
explained var:  0.009 non_lin: 1 n_components 81
explained var:  0.035 non_lin: 1 n_components 1
explained var:  0.032 non_lin: 1 n_components 61
explained var:  0.029 non_lin: 1 n_components 1
explained var:  0.03 non_lin: 1 n_components 41
explained var:  0.024 non_lin: 1 n_components 21
explained var:  0.047 non_lin: 1 n_components 81
explained var:  0.022 non_lin: 1 n_components 41
explained var:  0.038 non_lin: 1 n_components 81
explained var:  0.046 non_lin: 1 n_components 41
explained var:  0.021 non_lin: 1 n_components 61
explained var:  0.034 non_lin: 1 n_components 61
explained var:  0.032 non_lin: 1 n_components 41
explained var:  0.037 non_lin: 1 n_components 81
explained var:  0.041 n

In [9]:
output = open('./boc/formatted/LN_RS_Relu_Sigmoid', 'wb')
pickle.dump((best_model, scores_dict), output)
pkl_file.close()